In [228]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [229]:
# SCRIPT TO GET A DATAFRAME OF RANKINGS OF TOP X TEAMS, BY NCAA RANKING

LOWER_BOUND = 100 # the number of teams we get data from

rankings_page = requests.get('https://www.ncaa.com/rankings/basketball-men/d1/ncaa-mens-basketball-net-rankings')
rankings = BeautifulSoup(rankings_page.text, 'html5lib')
table = rankings.select('.rankings-content > table')[0]
table_rows = table.find_all('tr')
table_column_headers = table.find_all('th')
for i in range(len(table_column_headers)):
    table_column_headers[i] = table_column_headers[i].text
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
data = pd.DataFrame(l, columns=table_column_headers)
data.set_index('School',inplace=True)
data = data.iloc[1:LOWER_BOUND + 1]
print(f'finished downloading rankings of top {LOWER_BOUND} teams')

finished downloading rankings of top 100 teams


In [230]:
# IDS FOR PAGES OF TEAM STATS NCAA TRACKS
ids = [168,217,931,519,616,609,602,606,622,152,153,518,215,145,147,146,151,857,286,150,633,638,149,148,640,642,859,214,216,474,625]

In [231]:
# LOAD DATA FROM EACH TEAM STAT INTO A DATAFRAME, AND JOIN TO data
for id in ids:
    
    l = [] # STORES ROWS OF DATA
    name = None # OBJ TO LOOK FOR CURRENT STAT BEING TRACKED
    
    for i in range(1,8): # GOING THROUGH ALL 7 PAGES
        
        page = requests.get(f'https://www.ncaa.com/stats/basketball-men/d1/current/team/{id}/p{i}')
        soup = BeautifulSoup(page.text, 'html5lib')
        table = soup.select('.stats-wrap > table')[0] # SELECT TABLE
        
        if i == 1: # GETS TABLE HEADINGS (IF ON FIRST PAGE)
            name = soup.select('.stats-header__lower__title')[0]
            col_headers = table.find_all('th')
            for i in range(len(col_headers)):
                col_headers[i] = col_headers[i].text
        
        table_rows = table.find_all('tr') # GET ALL ROWS OF TABLE
        for tr in table_rows: # ADD DATA OF ROW INTO L
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
    
    data_stats = pd.DataFrame(l, columns=col_headers) # CREATE A DATAFRAME FOR CURRENT STAT
    data_stats.set_index('Team', inplace=True) # CHANGE INDEX TO TEAM NAME
    data_stats = data_stats[data_stats.index.isin(data.index.tolist())].drop(['Rank'],axis=1) # LIMIT IT TO THE TEAMS IN TOP LOWER_BOUND
    data = pd.concat([data,data_stats], axis=1, join='outer', sort=True) # JOIN CURRENT DF TO 'MASTER' DF
    
    
    name_title = name.text.lower().replace(' ', '-') # CLEANING NAME FOR PRINTING
    print(f'finished processing {name_title} ({id})')

finished processing won-lost-percentage (168)
finished processing turnovers-per-game (217)
finished processing turnovers-forced (931)
finished processing turnover-margin (519)
finished processing total-steals (616)
finished processing total-blocks (609)
finished processing total-rebounds (602)
finished processing total-assists (606)
finished processing total-3-point-fgm (622)
finished processing three-point-field-goal-percentage (152)
finished processing three-point-field-goals-per-game (153)
finished processing three-pt-fg-defense (518)
finished processing steals-per-game (215)
finished processing scoring-offense (145)
finished processing scoring-margin (147)
finished processing scoring-defense (146)
finished processing rebound-margin (151)
finished processing offensive-rebounds-per-game (857)
finished processing personal-fouls-per-game (286)
finished processing free-throw-percentage (150)
finished processing free-throws-made (633)
finished processing free-throw-attempts (638)
finishe

In [232]:
# CLEANING AND SAVING DATA
data = data.loc[:,~data.columns.duplicated()]
data['Rank'] = data['Rank'].astype(int)
data = data.sort_values(by=['Rank'])
data.to_csv(f'march-madness-data-top-{LOWER_BOUND}.csv')
print(f'finished cleaning, and saved data on the top {LOWER_BOUND} teams to csv')

finished cleaning, and saved data on the top 100 teams to csv
